<a href="https://colab.research.google.com/github/marisbotero/ML/blob/master/%F0%9F%A4%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np

In [0]:
options = ["👊", "✌", "🖐"]

In [0]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    
    elif p1 == "👊" and p2 == "✌":
        result = 1
    elif p1 == "👊" and p2 == "🖐":
        result = 2
    elif p1 == "✌" and p2 == "👊":
        result = 2
    elif p1 == "✌" and p2 == "🖐":
        result = 1
    elif p1 == "🖐" and p2 == "👊":
        result = 1
    elif p1 == "🖐" and p2 == "✌":
        result = 2
        
    return result

In [0]:
search_winner("🖐", "✌")

2

In [0]:
search_winner("✌","✌")

0

In [0]:
test = [
    ["👊", "👊", 0],
    ["👊", "✌", 1],
    ["👊", "🖐", 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

player1: 👊 player2: 👊 Winner: 0 Validation: 0
player1: 👊 player2: ✌ Winner: 1 Validation: 1
player1: 👊 player2: 🖐 Winner: 2 Validation: 2


In [0]:
from random import choice
def get_choice():
    return choice(options)

In [0]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: 👊 player2: 👊 Winner: 0 
player1: 👊 player2: ✌ Winner: 1 
player1: 🖐 player2: ✌ Winner: 2 
player1: ✌ player2: 🖐 Winner: 1 
player1: 👊 player2: ✌ Winner: 1 
player1: ✌ player2: 🖐 Winner: 1 
player1: 🖐 player2: 👊 Winner: 1 
player1: 👊 player2: 🖐 Winner: 2 
player1: 🖐 player2: ✌ Winner: 2 
player1: 👊 player2: 👊 Winner: 0 


In [0]:
def str_to_list(option):
    if option=="👊":
        res = [1,0,0]
    elif option=="✌":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, ["👊", "✌", "🖐"]))
data_y = list(map(str_to_list, ["🖐", "👊", "✌"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [0]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(verbose=False, warm_start=True)

In [0]:
model = clf.fit([data_X[0]], [data_y[0]])
print(model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=True)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        player1 = get_choice()
        
        predict = model.predict_proba([str_to_list(player1)])[0]
        
        if predict[0] >= 0.95:
            player2 = options[0]
        elif predict[1] >= 0.95:
            player2 = options[1]
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y

In [0]:
score, data_X, data_y = play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)

Player1: ✌ Player2 (modelo): [0.17561242 0.16144559 0.83639906] --> 👊
Comprobamos: p1 VS p2: 2
[[0, 1, 0]]
[[1, 0, 0]]
Score: {'win': 1, 'loose': 0} 100.0 %


In [0]:
i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historic_pct[-9:])==900:
        break

Iter: 1 - score: {'win': 341, 'loose': 659} 34.1 %
Iter: 2 - score: {'win': 317, 'loose': 683} 31.7 %
Iter: 3 - score: {'win': 366, 'loose': 634} 36.6 %
Iter: 4 - score: {'win': 326, 'loose': 674} 32.6 %
Iter: 5 - score: {'win': 333, 'loose': 667} 33.3 %
Iter: 6 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 7 - score: {'win': 340, 'loose': 660} 34.0 %
Iter: 8 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 9 - score: {'win': 329, 'loose': 671} 32.9 %
Iter: 10 - score: {'win': 326, 'loose': 674} 32.6 %
Iter: 11 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 12 - score: {'win': 325, 'loose': 675} 32.5 %
Iter: 13 - score: {'win': 340, 'loose': 660} 34.0 %
Iter: 14 - score: {'win': 345, 'loose': 655} 34.5 %
Iter: 15 - score: {'win': 304, 'loose': 696} 30.4 %
Iter: 16 - score: {'win': 351, 'loose': 649} 35.1 %
Iter: 17 - score: {'win': 349, 'loose': 651} 34.9 %
Iter: 18 - score: {'win': 353, 'loose': 647} 35.3 %
Iter: 19 - score: {'win': 336, 'loose': 664} 33.6 %
Iter: 20 - score: {'w

In [0]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook

In [0]:
x = range(len(historic_pct))
y = historic_pct

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iter", y_axis_label="%", width=900)

p.line(x, y, line_width=1)
show(p)

In [0]:
model.predict_proba([str_to_list("👊")])

array([[0.00520834, 0.00250728, 0.99147459]])